In [ ]:
from torch.utils import data

In [ ]:
# torch.utils.data.Dataset
# 可自定义数据集
# custom_dataset = CustomDataset() 
# CustomDataset(torch.utils.data.Dataset), 需定义好__init__(self), __getitem__(self, index)和__len__(self)

In [ ]:
train_set = None
train_set = data.Subset(train_set, train_idx_normal=[])

In [ ]:
dataset = None
train_loader = data.DataLoader(dataset, 4) # 以简单的方式提供队列和线程
# 第一个参数类型为torch.utils.data.Dataset, 第二个参数为batch_size
# 还有参数num_workers, shuffle(训练集置为True), pin_memory
	# 以及batch_sampler, collate_fn, sampler(当该参数为空时, 一般pin_memory=False)
# pin_memory 
	# torch.utils.data.DataLoader中尽量设置pin_memory=True
	# 对特别小的数据集如MNIST设置pin_memory=False反而更快一些
# num_workers
	# 设置需要在实验中找到最快的取值
data_iter = iter(train_loader) # 当迭代开始时, 队列和线程开始从文件下载数据
images, labels = data_iter.next() # Mini-batch

In [ ]:
# torch.utils.data.sampler.Sampler
# 一个迭代器基类
	# def __init__(self, data_source):
    #     pass

    # def __iter__(self):
    #     raise NotImplementedError

    # def __len__(self):
    #     raise NotImplementedError
# SequentialSampler
	# 按顺序对数据集采样
	# 在__iter__方法中首先得到一个和data_source一样长度的range可迭代器; 每次只会返回一个索引值
	# 即iter(range(len(self.data_source)))
# RandomSampler: 
	# num_samples: 指定采样的数量, 默认是所有
	# replacement: 若为True, 则表示可以重复采样, 即同一个样本可以重复采样, 这样可能导致有的样本采样不到; 所以此时我们可以设置num_samples来增加采样数量使得每个样本都可能被采样到
	# __iter__方法
		# n = len(self.data_source)
		# if self.replacement:
		# 	return iter(torch.randint(high=n, size=(self.num_samples,), dtype=torch.int64).tolist())
		# return iter(torch.randperm(n).tolist())
# SubsetRandomSampler
	# 这个采样器常见的使用场景是将训练集划分成训练集和验证集 
	# init函数中给定indices
	# iter函数返回(self.indices[i] for i in torch.randperm(len(self.indices)))
# WeightedRandomSampler
	# init函数中多了参数weights
	# self.weights = torch.tensor(weights, dtype=torch.double) 或 self.weights = torch.DoubleTensor(weights)
	# iter函数返回iter(torch.multinomial(self.weights, self.num_samples, self.replacement=True).tolist())